In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

In [2]:
df_monthly_infection = pd.read_excel("./data/Monthly_Infection_2001-2024.xlsx")
df_monthly_death = pd.read_excel("./data/Monthly_Death_2019-2024.xlsx")

In [3]:
df_monthly_infection

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Infected
0,2001,0,0,0,0,5,152,310,24,655,514,159,611,2430
1,2002,0,0,0,0,0,566,3281,0,1713,121,10,541,6232
2,2003,0,0,0,2,1,4,4,0,3,372,100,0,486
3,2004,0,0,1,0,1,1209,1261,143,511,395,51,362,3934
4,2005,1,0,0,0,0,3,174,2,337,187,19,325,1048
5,2006,0,0,0,0,0,972,680,100,211,92,0,145,2200
6,2007,0,0,0,0,0,85,179,0,163,0,0,39,466
7,2008,0,0,0,0,0,0,160,473,334,186,0,0,1153
8,2009,0,0,0,0,1,0,4,125,188,156,0,0,474
9,2010,0,0,0,0,0,0,61,183,120,45,0,0,409


In [20]:
month_num_map = {
    "January": "01",
    "February": "02",
    "March": "03",
    "April": "04",
    "May": "05",
    "June": "06",
    "July": "07",
    "August": "08",
    "September": "09",
    "October": "10",
    "November": "11",
    "December": "12",
}

months = list(month_num_map.keys())

In [21]:
df = df_monthly_infection

In [29]:
ordered_year_month = []
time_ordered_infections = []

for i in range(len(df)):
    curr_year = df.iloc[i]["Year"]
    curr_row = []

    for month in months:
        time_format = f"{curr_year}-{month_num_map[month]}"
        
        ordered_year_month.append(time_format)
        time_ordered_infections.append(df.iloc[i][month])

df_dict = {"YearMonth": ordered_year_month, "Infected": time_ordered_infections}

df_new = pd.DataFrame(df_dict)        

In [30]:
df_new

,YearMonth,Infected
0,2001-01,0
1,2001-02,0
2,2001-03,0
3,2001-04,0
4,2001-05,5
...,...,...
283,2024-08,6521
284,2024-09,18097
285,2024-10,30879
286,2024-11,29652


In [34]:
df_new.to_excel("./data/Dengue_Monthly_Infection_Cleaned_2001-2024.xlsx", index=False)